In [1]:
%env LANGCHAIN_TRACING_V2=true
%env LANGCHAIN_API_KEY=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
# The below examples use the OpenAI API, though it's not necessary in general
# %env OPENAI_API_KEY=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

env: LANGCHAIN_TRACING_V2=true
env: LANGCHAIN_API_KEY=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


In [2]:
from dotenv import load_dotenv
_ = load_dotenv()

In [2]:
import os

print(os.getenv("LANGCHAIN_API_KEY"))   

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


In [2]:
import os
from langchain_openai import ChatOpenAI

class ChatOpenRouter(ChatOpenAI):
    def __init__(self, model_name: str, openai_api_base: str = None, openai_api_key: str = None, **kwargs):
        openai_api_base = openai_api_base or "https://openrouter.ai/api/v1"
        openai_api_key = "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"
        super().__init__(
            openai_api_base=openai_api_base,
            openai_api_key=openai_api_key,
            model_name=model_name,
            **kwargs,
        )

In [ ]:
from typing import Literal

from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langgraph.graph import StateGraph, MessagesState
from langgraph.prebuilt import ToolNode
from langchain_groq import ChatGroq

@tool
def search(query: str):
    """Call to surf the web."""
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."

tools = [search]

tool_node = ToolNode(tools)

model = ChatGroq(name="llama3-70b-8192", temperature=0).bind_tools(tools)

def should_continue(state: MessagesState) -> Literal["tools", "__end__"]:
    messages = state['messages']
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return "__end__"


def call_model(state: MessagesState):
    messages = state['messages']

    # Invoking `model` will automatically infer the correct tracing context
    response = model.invoke(messages)
    return {"messages": [response]}


workflow = StateGraph(MessagesState)

workflow.add_node("agent", call_model)
workflow.add_node("tools", tool_node)

workflow.add_edge("__start__", "agent")
workflow.add_conditional_edges(
    "agent",
    should_continue,
)
workflow.add_edge("tools", 'agent')

app = workflow.compile()

final_state = app.invoke(
    {"messages": [HumanMessage(content="what is the weather in sf")]},
    config={"configurable": {"thread_id": 42}}
)
final_state["messages"][-1].content

{'messages': [HumanMessage(content='what is the weather in sf', additional_kwargs={}, response_metadata={}, id='68bf4f4e-841e-4000-ba09-01b1f5fd63be'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ca4e', 'function': {'arguments': '{"query":"weather in sf"}', 'name': 'search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 90, 'prompt_tokens': 1158, 'total_tokens': 1248, 'completion_time': 0.138129611, 'prompt_time': 0.050602691, 'queue_time': 0.022400335, 'total_time': 0.188732302}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-af512c6f-cf0b-40ce-a0d8-f1a76e36bf42-0', tool_calls=[{'name': 'search', 'args': {'query': 'weather in sf'}, 'id': 'call_ca4e', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1158, 'output_tokens': 90, 'total_tokens': 1248}),
  ToolMessage(content="It's 60 degrees and foggy.", name='search', id='bca4bd7a-c84a-46